In [1]:
import numpy as np
import matplotlib.pyplot as plt
import Image
import subprocess

In [3]:
import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne

from collections import OrderedDict
from lasagne import utils

In [4]:
def make_seed(n):
    np.random.seed(n)
    lasagne.random.set_rng(np.random)

In [5]:
# lasagne.random.set_rng(np.random)
print(lasagne.random.get_rng())

<module 'numpy.random' from '/usr/local/lib/python2.7/dist-packages/numpy/random/__init__.pyc'>


In [2]:
subprocess.call(['th', '-l', 'my_example'])

0

In [6]:
def adam_update(loss_or_grads, params, learning_rate=1e-3, beta1=0.9,
                        beta2=0.999, epsilon=1e-8):
    all_grads = lasagne.updates.get_or_compute_grads(loss_or_grads, params)
    t_prev = theano.shared(utils.floatX(0.))
    updates = OrderedDict()

    t = t_prev + 1
    a_t = learning_rate * T.sqrt(1 - beta2 ** t) / (1 - beta1 ** t)

    for param, g_t in zip(params, all_grads):
        value = param.get_value(borrow=True)
        m_prev = theano.shared(np.zeros(value.shape, dtype=value.dtype),
                               broadcastable=param.broadcastable)
        v_prev = theano.shared(np.zeros(value.shape, dtype=value.dtype),
                               broadcastable=param.broadcastable)

        m_t = beta1 * m_prev + (1 - beta1) * g_t
        v_t = beta2 * v_prev + (1 - beta2) * g_t ** 2
        step = a_t * m_t / (T.sqrt(v_t) + epsilon)

        updates[m_prev] = m_t
        updates[v_prev] = v_t
        updates[param] = param - step

    updates[t_prev] = t
    return updates


def adam_update2(loss_or_grads, params, learning_rate=1e-3, beta1=0.9,
                 beta2=0.999, epsilon=1e-6):
    all_grads = lasagne.updates.get_or_compute_grads(loss_or_grads, params)


    t_prev = theano.shared(utils.floatX(0.))
    updates = OrderedDict()

    t = t_prev + 1
    a_t = learning_rate * T.sqrt(1 - beta2 ** t) / (1 - beta1 ** t)

    for param, g_t in zip(params, all_grads):
        value = param.get_value(borrow=True)
        m_prev = theano.shared(np.zeros(value.shape, dtype=value.dtype),
                               broadcastable=param.broadcastable)
        v_prev = theano.shared(np.zeros(value.shape, dtype=value.dtype),
                               broadcastable=param.broadcastable)

        m_t = beta1 * m_prev + (1 - beta1) * g_t
        v_t = beta2 * v_prev + (1 - beta2) * g_t ** 2
        step = a_t * m_t / (T.sqrt(v_t + epsilon))

        updates[m_prev] = m_t
        updates[v_prev] = v_t
        updates[param] = param - step

    updates[t_prev] = t
    return updates

In [7]:
import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne

In [8]:
def build_mlp(input_var=None, BN=False):
    l_in = lasagne.layers.InputLayer(shape=(None, 1, sz, sz),
                                     input_var=input_var)
    if BN:
        l_in = lasagne.layers.batch_norm(l_in)

    l_hid1 = lasagne.layers.DenseLayer(
            l_in, num_units=100,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())
    if BN:
        l_hid1 = lasagne.layers.batch_norm(l_hid1)

    l_hid2 = lasagne.layers.DenseLayer(
            l_hid1, num_units=100,
            nonlinearity=lasagne.nonlinearities.rectify)
    if BN:
        l_hid2 = lasagne.layers.batch_norm(l_hid2)
        
    l_hid3 = lasagne.layers.DenseLayer(
            l_hid2, num_units=100,
            nonlinearity=lasagne.nonlinearities.rectify)
    if BN:
        l_hid3 = lasagne.layers.batch_norm(l_hid3)

    l_out = lasagne.layers.DenseLayer(
            l_hid3, num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax)
    return l_out

In [9]:
def get_data():
    subprocess.call(['th', '-l', 'my_example'])
    n, d = 10000, 3600
    train_images = np.zeros((n, d), dtype=np.uint8)
    train_labels = np.zeros(n, dtype=np.uint8)
    for i in range(n):
        image_open = Image.open("clMNIST/example" + str(i) + ".png")
        a = np.array(image_open.getdata())
        train_images[i] = a 
        train_labels[i] = np.uint(np.loadtxt("clMNIST/y" + str(i)))
    return np.reshape(train_images, (-1, 1, 60, 60)), np.ravel(train_labels)

In [10]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [11]:
sz = 60

In [12]:
def run_method(method, model='mlp', BN=False, num_epochs=50, alpha=0.1, mu=0.9, beta1=0.9, beta2=0.999, echo=False, 
               batch_size=100):
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    if echo:
        print("Building model and compiling functions...")
    if model == 'mlp':
        network = build_mlp(input_var, BN)
    else:
        print("Unrecognized model type %r." % model)
        return

    prediction = lasagne.layers.get_output(network)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()
    train_acc = T.mean(T.eq(T.argmax(prediction, axis=1), target_var),
                 dtype=theano.config.floatX)

    params = lasagne.layers.get_all_params(network, trainable=True)

    if method == lasagne.updates.sgd:
        updates = method(loss, params, learning_rate=alpha)
    elif method == lasagne.updates.momentum:
        updates = method(loss, params, learning_rate=alpha, momentum=mu)
    elif method == lasagne.updates.adam:
        updates = method(loss, params, learning_rate=alpha, beta1=beta1)
    elif method == adam_update or method == adam_update2:
        updates = method(loss, params, learning_rate=alpha, beta1=beta1, beta2=beta2)
    else:
        updates = method(loss, params, learning_rate=alpha)


    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)
    train_fn_acc = theano.function([input_var, target_var], train_acc)
    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    if echo:
        print("Starting training...")

    res = dict()
    arr_train_err = []
    arr_val_err = []
    arr_train_acc = []
    arr_val_acc = []

    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        train_acc = 0
        start_time = time.time()
        
        train_inputs, train_targets = get_data()
        
        for batch in iterate_minibatches(train_inputs, train_targets, batch_size, shuffle=True):
            inputs, targets = batch
            err = train_fn(inputs, targets)
            acc = train_fn_acc(inputs, targets)
            train_err += err
            train_acc += acc
            train_batches += 1

        arr_train_err.append(train_err / train_batches)
        arr_train_acc.append(train_acc / train_batches * 100)
        
        val_inputs, val_targets = get_data()

        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(val_inputs, val_targets, batch_size, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1

        arr_val_err.append(val_err / val_batches)
        arr_val_acc.append(val_acc / val_batches * 100)

        if echo:
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, num_epochs, time.time() - start_time))
            print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
#             print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
#             print("  validation accuracy:\t\t{:.2f} %".format(
#                 val_acc / val_batches * 100))

    test_err = 0
    test_acc = 0
    test_batches = 0
    
    test_inputs, test_targets = get_data()
        
    for batch in iterate_minibatches(test_inputs, test_targets, batch_size, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        test_err += err
        test_acc += acc
        test_batches += 1

    if echo:
        print("Final results:")
        print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
        print("  test accuracy:\t\t{:.2f} %".format(
            test_acc / test_batches * 100))

    res['train_err'] = np.array(arr_train_err)
    res['val_err'] = np.array(arr_val_err)
    res['train_acc'] = np.array(arr_train_acc)
    res['val_acc'] = np.array(arr_val_acc)
    res['test_err'] = test_err / test_batches
    res['test_acc'] = test_acc / test_batches * 100

    return res

In [109]:
%%time
make_seed()
adam1_999 = run_method(adam_update, num_epochs=50, alpha=1e-3, beta2=0.999)

CPU times: user 17min 33s, sys: 14min 58s, total: 32min 31s
Wall time: 22min 42s


In [110]:
%%time
make_seed()
bn_adam1_999 = run_method(adam_update, num_epochs=50, alpha=1e-3, BN=True, beta2=0.999)

CPU times: user 20min 10s, sys: 22min 28s, total: 42min 38s
Wall time: 24min 12s


In [160]:
import pickle

In [163]:
with open("different_beta2/adam1", "wb") as file1:
    pickle.dump(adam1_999, file1)
    pickle.dump(bn_adam1_999, file1)
    pickle.dump(adam1_99, file1)
    pickle.dump(bn_adam1_99, file1)
    pickle.dump(adam1_9, file1)
    pickle.dump(bn_adam1_9, file1)

In [111]:
%%time
make_seed()
adam1_99 = run_method(adam_update, num_epochs=50, alpha=1e-3, beta2=0.99)

CPU times: user 17min 23s, sys: 15min 3s, total: 32min 27s
Wall time: 22min 38s


In [112]:
%%time
make_seed()
bn_adam1_99 = run_method(adam_update, num_epochs=50, alpha=1e-3, BN=True, beta2=0.99)

CPU times: user 20min 15s, sys: 22min 3s, total: 42min 19s
Wall time: 24min 8s


In [113]:
%%time
make_seed()
adam1_9 = run_method(adam_update, num_epochs=50, alpha=1e-3, beta2=0.9)

CPU times: user 17min 35s, sys: 15min 32s, total: 33min 8s
Wall time: 22min 45s


In [114]:
%%time
make_seed()
bn_adam1_9 = run_method(adam_update, num_epochs=50, alpha=1e-3, BN=True, beta2=0.9)

CPU times: user 20min 15s, sys: 22min 21s, total: 42min 36s
Wall time: 24min 12s


In [115]:
%%time
make_seed()
adam2_999 = run_method(adam_update2, num_epochs=50, alpha=1e-3, beta2=0.999)

CPU times: user 17min 43s, sys: 15min 34s, total: 33min 17s
Wall time: 22min 45s


In [116]:
%%time
make_seed()
bn_adam2_999 = run_method(adam_update2, num_epochs=50, alpha=1e-3, BN=True, beta2=0.999)

CPU times: user 20min 20s, sys: 22min 30s, total: 42min 51s
Wall time: 24min 14s


In [117]:
%%time
make_seed()
adam2_99 = run_method(adam_update2, num_epochs=50, alpha=1e-3, beta2=0.99)

CPU times: user 17min 44s, sys: 16min 7s, total: 33min 51s
Wall time: 22min 51s


In [118]:
%%time
make_seed()
bn_adam2_99 = run_method(adam_update2, num_epochs=50, alpha=1e-3, BN=True, beta2=0.99)

CPU times: user 20min 28s, sys: 23min 16s, total: 43min 45s
Wall time: 24min 20s


In [119]:
%%time
make_seed()
adam2_9 = run_method(adam_update2, num_epochs=50, alpha=1e-3, beta2=0.9)

CPU times: user 17min 43s, sys: 15min 58s, total: 33min 42s
Wall time: 22min 51s


In [120]:
%%time
make_seed()
bn_adam2_9 = run_method(adam_update2, num_epochs=50, alpha=1e-3, BN=True, beta2=0.9)

CPU times: user 20min 27s, sys: 22min 26s, total: 42min 53s
Wall time: 24min 19s


In [164]:
with open("different_beta2/adam2", "wb") as file1:
    pickle.dump(adam2_999, file1)
    pickle.dump(bn_adam2_999, file1)
    pickle.dump(adam2_99, file1)
    pickle.dump(bn_adam2_99, file1)
    pickle.dump(adam2_9, file1)
    pickle.dump(bn_adam2_9, file1)

In [166]:
plt.plot(bn_adam1_9['val_acc'])
plt.show()

### ==========================================================

In [96]:
%%time
sgd = run_method(lasagne.updates.sgd, batch_num=100, num_epochs=50, alpha=0.1)

CPU times: user 11min 45s, sys: 4min 43s, total: 16min 29s
Wall time: 46min 23s


In [97]:
%%time
bn_sgd = run_method(lasagne.updates.sgd, batch_num=100, num_epochs=50, BN=True, alpha=1.0)

CPU times: user 9min 58s, sys: 4min 50s, total: 14min 48s
Wall time: 34min 32s


In [116]:
import pickle
with open("infinite_clmnist_sgd", "wb") as file1:
    pickle.dump(sgd, file1)
    pickle.dump(bn_sgd, file1)

In [123]:
import pickle
with open("infinite_clmnist_sgd", "rb") as file1:
    qa = pickle.load(file1)
    qb = pickle.load(file1)

In [98]:
%%time
adam = run_method(lasagne.updates.adam, batch_num=100, num_epochs=50, alpha=0.001)

CPU times: user 8min 55s, sys: 6min 11s, total: 15min 6s
Wall time: 29min 4s


In [102]:
%%time
bn_adam = run_method(lasagne.updates.adam, batch_num=100, num_epochs=50, alpha=0.001, BN=True)

INFO (theano.gof.compilelock): Refreshing lock /root/.theano/compiledir_Linux-3.13--generic-x86_64-with-Ubuntu-14.04-trusty-x86_64-2.7.6-64/lock_dir/lock


CPU times: user 10min 8s, sys: 7min 55s, total: 18min 4s
Wall time: 29min 39s


In [117]:
import pickle
with open("infinite_clmnist_adam", "wb") as file1:
    pickle.dump(adam, file1)
    pickle.dump(bn_adam, file1)

In [131]:
import pickle
with open("infinite_clmnist_adam", "rb") as file1:
    qa = pickle.load(file1)
    qb = pickle.load(file1)

In [112]:
plt.plot(sgd['train_acc'], 'b')
plt.plot(bn_sgd['train_acc'], 'r')
plt.legend(['SGD', 'BN SGD'], loc=0, fontsize=12)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.title('Infinite clMNIST, SGD')
plt.show()

/usr/local/lib/python2.7/dist-packages/matplotlib/backends/backend_gtk3.py:725: Warning: g_value_set_object: assertion 'G_VALUE_HOLDS_OBJECT (value)' failed
  if self.run() != int(Gtk.ResponseType.OK):


In [115]:
plt.plot(adam['train_acc'], 'b')
plt.plot(bn_adam['train_acc'], 'r')
plt.legend(['Adam', 'BN Adam'], loc=0, fontsize=12)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.title('Infinite clMNIST, Adam')
plt.show()

In [134]:
plt.plot(bn_sgd['train_acc'], 'r--')
plt.plot(bn_adam['train_acc'], 'r')
plt.legend(['BN SGD', 'BN Adam'], loc=0, fontsize=12)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.title('Infinite clMNIST')
plt.show()

In [135]:
plt.plot(sgd['train_acc'], 'b--')
plt.plot(adam['train_acc'], 'b')
plt.legend(['SGD', 'Adam'], loc=0, fontsize=12)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.title('Infinite clMNIST')
plt.show()

### ========================================================================

In [16]:
%%time
make_seed(1)
adam1_999 = run_method(adam_update, num_epochs=50, alpha=1e-3, beta2=0.999)

CPU times: user 17min 45s, sys: 16min 54s, total: 34min 40s
Wall time: 22min 58s


In [17]:
%%time
make_seed(1)
bn_adam1_999 = run_method(adam_update, num_epochs=50, alpha=1e-3, BN=True, beta2=0.999)

CPU times: user 20min 38s, sys: 25min 35s, total: 46min 13s
Wall time: 24min 45s


In [18]:
%%time
make_seed(10)
adam10_999 = run_method(adam_update, num_epochs=50, alpha=1e-3, beta2=0.999)

CPU times: user 17min 57s, sys: 19min 38s, total: 37min 36s
Wall time: 23min 21s


In [19]:
%%time
make_seed(10)
bn_adam10_999 = run_method(adam_update, num_epochs=50, alpha=1e-3, BN=True, beta2=0.999)

CPU times: user 20min 46s, sys: 25min 10s, total: 45min 56s
Wall time: 24min 46s


In [20]:
%%time
make_seed(100)
adam100_999 = run_method(adam_update, num_epochs=50, alpha=1e-3, beta2=0.999)

CPU times: user 17min 51s, sys: 16min 43s, total: 34min 34s
Wall time: 23min 7s


In [21]:
%%time
make_seed(100)
bn_adam100_999 = run_method(adam_update, num_epochs=50, alpha=1e-3, BN=True, beta2=0.999)

CPU times: user 20min 42s, sys: 24min 44s, total: 45min 26s
Wall time: 24min 52s


In [22]:
%%time
make_seed(1)
adam1_99 = run_method(adam_update, num_epochs=50, alpha=1e-3, beta2=0.99)

CPU times: user 17min 53s, sys: 18min 1s, total: 35min 55s
Wall time: 23min 25s


In [23]:
%%time
make_seed(1)
bn_adam1_99 = run_method(adam_update, num_epochs=50, alpha=1e-3, BN=True, beta2=0.99)

CPU times: user 20min 45s, sys: 24min 39s, total: 45min 25s
Wall time: 24min 57s


In [24]:
%%time
make_seed(10)
adam10_99 = run_method(adam_update, num_epochs=50, alpha=1e-3, beta2=0.99)

CPU times: user 17min 51s, sys: 16min 37s, total: 34min 29s
Wall time: 23min 17s


In [25]:
%%time
make_seed(10)
bn_adam10_99 = run_method(adam_update, num_epochs=50, alpha=1e-3, BN=True, beta2=0.99)

CPU times: user 20min 47s, sys: 24min 58s, total: 45min 45s
Wall time: 25min


In [26]:
%%time
make_seed(100)
adam100_99 = run_method(adam_update, num_epochs=50, alpha=1e-3, beta2=0.99)

CPU times: user 17min 49s, sys: 16min 31s, total: 34min 21s
Wall time: 23min 16s


In [27]:
%%time
make_seed(100)
bn_adam100_99 = run_method(adam_update, num_epochs=50, alpha=1e-3, BN=True, beta2=0.99)

CPU times: user 20min 33s, sys: 24min 49s, total: 45min 22s
Wall time: 24min 56s


In [47]:
%%time
make_seed(1)
adam1_9 = run_method(adam_update, num_epochs=50, alpha=1e-3, beta2=0.9)

CPU times: user 18min, sys: 16min 52s, total: 34min 52s
Wall time: 23min 24s


In [48]:
%%time
make_seed(1)
bn_adam1_9 = run_method(adam_update, num_epochs=50, alpha=1e-3, BN=True, beta2=0.9)

CPU times: user 20min 59s, sys: 26min, total: 47min
Wall time: 25min 34s


In [63]:
import pickle
with open("different_beta2/many_adam", "wb") as file1:
    pickle.dump(adam1_999, file1)
    pickle.dump(bn_adam1_999, file1)
    
    pickle.dump(adam10_999, file1)
    pickle.dump(bn_adam10_999, file1)
    
    pickle.dump(adam100_999, file1)
    pickle.dump(bn_adam100_999, file1)
    
    pickle.dump(adam1_99, file1)
    pickle.dump(bn_adam1_99, file1)
    
    pickle.dump(adam10_99, file1)
    pickle.dump(bn_adam10_99, file1)
    
    pickle.dump(adam100_99, file1)
    pickle.dump(bn_adam100_99, file1)
    
    pickle.dump(adam1_9, file1)
    pickle.dump(bn_adam1_9, file1)

In [16]:
import pickle
with open("different_beta2/many_adam", "rb") as file1:
    adam1_999 = pickle.load(file1)
    bn_adam1_999 = pickle.load(file1)
    
    adam10_999 = pickle.load(file1)
    bn_adam10_999 = pickle.load(file1)
    
    adam100_999 = pickle.load(file1)
    bn_adam100_999 = pickle.load(file1)
    
    adam1_99 = pickle.load(file1)
    bn_adam1_99 = pickle.load(file1)
    
    adam10_99 = pickle.load(file1)
    bn_adam10_99 = pickle.load(file1)
    
    adam100_99 = pickle.load(file1)
    bn_adam100_99 = pickle.load(file1)
    
    adam1_9 = pickle.load(file1)
    bn_adam1_9 = pickle.load(file1)

In [47]:
plt.plot(adam1_9['train_acc'], 'b--')
plt.plot(adam1_9['val_acc'], 'b')
plt.plot(bn_adam1_9['train_acc'], 'r--')
plt.plot(bn_adam1_9['val_acc'], 'r')
plt.show()

In [27]:
%%time
make_seed(1)
bn_adam_another = run_method(adam_update2, num_epochs=50, alpha=1e-3, beta2=0.9, BN=True)

CPU times: user 20min, sys: 14min 20s, total: 34min 21s
Wall time: 20min 48s


In [29]:
%%time
make_seed(10)
bn_adam_another10 = run_method(adam_update2, num_epochs=50, alpha=1e-3, beta2=0.9, BN=True)

CPU times: user 20min 21s, sys: 15min 22s, total: 35min 43s
Wall time: 21min 40s


In [30]:
%%time
make_seed(100)
bn_adam_another100 = run_method(adam_update2, num_epochs=50, alpha=1e-3, beta2=0.9, BN=True)

CPU times: user 20min 3s, sys: 14min 26s, total: 34min 29s
Wall time: 20min 52s


In [31]:
%%time
make_seed(1)
bn_adagrad1 = run_method(lasagne.updates.adagrad, num_epochs=50, BN=True)

CPU times: user 19min 32s, sys: 12min 10s, total: 31min 43s
Wall time: 20min 33s


In [32]:
%%time
make_seed(1)
bn_adadelta1 = run_method(lasagne.updates.adadelta, num_epochs=50, BN=True)

CPU times: user 19min 29s, sys: 17min 41s, total: 37min 11s
Wall time: 21min 17s


In [49]:
plt.plot(bn_adadelta1['train_acc'], 'b--')
plt.plot(bn_adadelta1['val_acc'], 'b')
plt.show()

In [46]:
plt.plot(bn_adam_another100['train_acc'], 'r--')
plt.plot(bn_adam_another100['val_acc'], 'r')
plt.show()

### RMSProp

In [13]:
%%time
make_seed(1)
rsmprop3 = run_method(lasagne.updates.rmsprop, num_epochs=50, alpha=1e-3)

CPU times: user 16min 34s, sys: 7min 6s, total: 23min 40s
Wall time: 19min 28s


In [14]:
%%time
make_seed(1)
bn_rsmprop3 = run_method(lasagne.updates.rmsprop, BN=True, num_epochs=50, alpha=1e-3)

CPU times: user 20min 2s, sys: 16min 18s, total: 36min 20s
Wall time: 23min 10s


In [51]:
plt.plot(rsmprop3['train_acc'], 'b--')
plt.plot(rsmprop3['val_acc'], 'b')
plt.plot(bn_rsmprop3['train_acc'], 'r--')
plt.plot(bn_rsmprop3['val_acc'], 'r')
plt.show()

In [50]:
import pickle
with open("three_methods_vs_adam2", "wb") as file1:
    pickle.dump(bn_adam_another, file1)
    
    pickle.dump(bn_adam_another10, file1)
    
    pickle.dump(bn_adam_another100, file1)
    
    pickle.dump(bn_adagrad1, file1)
    
    pickle.dump(bn_adadelta1, file1)
    
    pickle.dump(rsmprop3, file1)
    pickle.dump(bn_rsmprop3, file1)